# 4장: 데이터와 지침 분리

- [수업](#lesson)
- [연습 문제](#exercises)
- [예시 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

종종 전체 프롬프트를 작성하고 싶지 않고 대신 **Claude에 제출하기 전에 추가 입력 데이터로 수정할 수 있는 프롬프트 템플릿을 원합니다**. 이것은 Claude가 매번 동일한 작업을 수행하기를 원하지만 Claude가 작업에 사용하는 데이터가 매번 다를 수 있는 경우에 유용할 수 있습니다.

다행히도 **프롬프트의 고정 골격과 변수 사용자 입력을 분리한 다음 Claude에 전체 프롬프트를 보내기 전에 사용자 입력을 프롬프트에 대체하여** 이를 꽤 쉽게 수행할 수 있습니다.

아래에서는 대체 가능한 프롬프트 템플릿을 작성하는 방법과 사용자 입력을 대체하는 방법을 단계별로 살펴볼 것입니다.

### 예시

첫 번째 예시에서는 Claude에게 동물 소리 생성기 역할을 요청하고 있습니다. Claude에 제출된 전체 프롬프트는 입력(이 경우 "Cow")으로 대체된 `PROMPT_TEMPLATE`임을 주목하세요. 전체 프롬프트를 출력할 때 "Cow"라는 단어가 f-string을 통해 `ANIMAL` 자리 표시자를 대체하는 것을 확인할 수 있습니다.

**참고:** 실제로 자리 표시자 변수를 특별히 부르지 않아도 됩니다. 이 예시에서는 `ANIMAL`이라고 불렀지만, 쉽게 `CREATURE` 또는 `A`라고 부를 수 있습니다(그러나 일반적으로 프롬프트 템플릿이 대체 없이도 사용자가 해석할 수 있도록 변수 이름을 구체적이고 관련성 있게 지정하는 것이 좋습니다). 프롬프트 템플릿 f-string에서 사용하는 것과 동일한 이름을 변수에 지정하기만 하면 됩니다.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

이렇게 입력을 분리하고 대체하는 이유는 무엇일까요? 프롬프트 템플릿은 **반복 작업을 단순화합니다**. 예를 들어 제3자 사용자가 프롬프트에 콘텐츠를 제출할 수 있도록 프롬프트 구조를 구축한다고 가정해 봅시다(이 경우 소리를 생성할 동물). 이러한 제3자 사용자는 전체 프롬프트를 작성하거나 볼 필요가 없습니다. 그들이 해야 할 일은 변수만 채우는 것입니다.

여기서는 변수와 f-string을 사용하여 대체를 수행하지만 format() 메서드를 사용할 수도 있습니다.

**참고:** 프롬프트 템플릿에는 원하는 만큼 많은 변수를 사용할 수 있습니다!

이렇게 대체 변수를 소개할 때 **Claude에게 변수가 시작하고 끝나는 위치를 알려주는 것이 매우 중요합니다**(지침이나 작업 설명과 구분). 지침과 대체 변수 사이에 구분이 없는 예를 살펴보겠습니다.

우리 인간의 눈으로는 아래 프롬프트 템플릿에서 변수가 시작하고 끝나는 위치가 매우 명확합니다. 그러나 완전히 대체된 프롬프트에서는 그 구분이 불분명해집니다.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

여기서 **Claude는 "Yo Claude"가 다시 작성해야 할 이메일의 일부라고 생각합니다**! 다시 작성을 "Dear Claude"로 시작하기 때문에 알 수 있습니다. 사람의 눈에는 이메일이 시작되고 끝나는 프롬프트 템플릿에서 명확하지만, 대체 후 프롬프트에서는 덜 명확해집니다.

이를 어떻게 해결할까요? **입력을 XML 태그로 감싸세요**! 아래에서 이렇게 했고, 출력에 더 이상 "Dear Claude"가 없는 것을 볼 수 있습니다.

[XML 태그](https://docs.anthropic.com/claude/docs/use-xml-tags)는 `<tag></tag>`와 같은 꺾쇠 괄호 태그입니다. 쌍으로 이루어져 있으며 `<tag>`와 같은 여는 태그와 `</tag>`와 같은 닫는 태그로 구성됩니다. XML 태그는 `<tag>content</tag>`와 같이 콘텐츠를 감싸는 데 사용됩니다.

**참고:** Claude는 다양한 구분 기호와 구분자를 인식하고 작업할 수 있지만, Claude가 프롬프트 구성 메커니즘으로 XML 태그를 인식하도록 특별히 훈련되었기 때문에 **Claude에 대한 구분자로 특별히 XML 태그를 사용하는 것이 좋습니다**. 함수 호출 외에는 **성능을 최대한 높이기 위해 Claude가 훈련된 특별한 XML 태그가 없습니다**. Anthropic은 의도적으로 Claude를 매우 유연하고 사용자 정의할 수 있도록 만들었습니다.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

XML 태그가 어떻게 도움이 되는지 다른 예를 살펴봅시다. 

다음 프롬프트에서 **Claude는 프롬프트의 어느 부분이 지시사항이고 입력인지를 잘못 해석합니다**. 사용자(SENTENCES 변수를 채우는 사람)가 원하지 않았을 것 같은데도 `Each is about an animal, like rabbits`를 목록의 일부로 잘못 간주합니다.

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

이를 해결하려면 **사용자 입력 문장을 XML 태그로 감싸기만 하면 됩니다**. 이렇게 하면 `Each is about an animal, like rabbits.` 앞의 하이픈이 혼란스럽더라도 Claude에게 입력 데이터가 시작되고 끝나는 위치를 보여줄 수 있습니다.

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**참고:** "Each is about an animal"이라는 프롬프트의 잘못된 버전에서는 이 예제에서 원하는 대로 Claude가 잘못 응답하게 하려면 하이픈을 포함해야 했습니다. 이것은 프롬프팅에 대한 중요한 교훈입니다: **작은 세부 사항이 중요합니다**! **프롬프트에서 오타와 문법 오류를 제거하는 것이 항상 중요합니다**. Claude는 패턴에 민감합니다(초기에는 미세조정 전 원시 텍스트 예측 도구였습니다). 실수를 하면 실수할 가능성이 더 높고, 똑똑하게 들리면 더 똑똑해지고, 바보 같이 들리면 더 바보 같아집니다.

위의 내용을 변경하지 않고 수업 프롬프트를 실험해 보고 싶다면 수업 노트북 맨 아래로 스크롤하여 [**예제 놀이터**](#example-playground)를 방문하세요.

---

## 연습 문제
- [연습문제 4.1 - 하이쿠 주제](#exercise-41---haiku-topic)
- [연습문제 4.2 - 오타가 있는 개 질문](#exercise-42---dog-question-with-typos)
- [연습문제 4.3 - 개 질문 파트 2](#exercise-42---dog-question-part-2)

### 연습 4.1 - 하이쿠 주제
변수 `TOPIC`을 받아 해당 주제에 대한 하이쿠를 출력하는 템플릿으로 `PROMPT`를 수정하세요. 이 연습은 f-string을 사용한 변수 템플릿 구조에 대한 이해를 테스트하기 위한 것입니다.

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f"Create a haiku about pigs"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_4_1_hint)

### 연습 4.2 - 오타가 있는 개 질문
XML 태그를 추가하여 `PROMPT`를 수정하고 Claude가 올바른 답변을 생성하도록 하세요.

프롬프트의 다른 부분은 변경하지 마세요. 엉망이고 실수 투성이인 글은 Claude가 그런 실수에 어떻게 반응하는지 보기 위한 의도적인 것입니다.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_4_2_hint)

### 연습 4.3 - 개 질문 파트 2
XML 태그를 추가하지 않고 `PROMPT`를 수정하세요. 대신 프롬프트에서 한두 단어만 제거하세요.

위의 연습과 마찬가지로 프롬프트의 다른 부분은 변경하지 마세요. 이를 통해 Claude가 어떤 종류의 언어를 구문 분석하고 이해할 수 있는지 알 수 있습니다.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_4_3_hint)

### 축하합니다!

지금까지의 모든 연습을 해결했다면 다음 장으로 넘어갈 준비가 되었습니다. 프롬프팅을 즐기세요!

---

## 예제 놀이터

이 영역에서는 이 수업에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있습니다.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))